<a href="https://colab.research.google.com/github/su92-msaiw-f25-016-spec/lessons-learned/blob/Machine_Learning_Project/Priceprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt